# Collaborative Filtering

## The Framework

In [ ]:
#!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip (movie, rating, users)

In [8]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2020-09-03 06:56:39--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  5.82MB/s    in 1.0s    

2020-09-03 06:56:40 (5.82 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [9]:
import zipfile
with zipfile.ZipFile("/content/ml-1m.zip","r") as zip_ref:
    zip_ref.extractall()

In [3]:
import pandas as pd
import numpy as np

In [6]:
#Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('/content/ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
#Load the u.item file into a dataframe
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('/content/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')
print(movies.shape)
movies.head()

(1682, 24)


,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [10]:
#Remove all information except Movie ID and title
movies = movies[['movie_id', 'title']]

In [12]:
#Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('/content/ml-100k/u.data', sep='\t', names=r_cols,
 encoding='latin-1')
print(ratings.shape)

ratings.head()

(100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
#Drop the timestamp column
ratings = ratings.drop('timestamp', axis=1)

In [14]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = ratings.copy()
y = ratings['user_id']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [15]:
#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [16]:
#Define the baseline model to always return 3.
def baseline(user_id, movie_id):
    return 3.0

In [17]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [18]:
score(baseline)

1.2470926188539486

## User Based Collaborative Filtering

### Ratings Matrix

In [19]:
#Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1647,1648,1649,1651,1652,1653,1654,1656,1657,1658,1659,1660,1661,1662,1663,1664,1668,1669,1670,1671,1673,1674,1675,1676,1679,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,NaN,5.0,NaN,NaN,NaN,4.0,5.0,4.0,1.0,NaN,4.0,3.0,4.0,3.0,NaN,4.0,1.0,NaN,3.0,5.0,4.0,NaN,1.0,2.0,NaN,3.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Mean

In [20]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(user_id, movie_id):
    
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[movie_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [21]:
#Compute RMSE for the Mean model
score(cf_user_mean)

1.0234701463131335

### Weighted Mean

In [22]:
#Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [23]:
# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [24]:
#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head(10)

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.118076,0.029097,0.011628,0.264677,0.312419,0.308729,0.224269,0.026017,0.286411,0.220437,0.213117,0.351204,0.235971,0.138522,0.341201,0.136597,0.340766,0.088945,0.168888,0.168084,0.245544,0.363621,0.244380,0.266342,0.184639,0.142040,0.218228,0.050924,0.149212,0.091746,0.129824,0.038110,0.011462,0.016556,0.000000,0.153244,0.148866,0.030900,0.038933,...,0.103657,0.109444,0.070591,0.217109,0.238477,0.126022,0.154188,0.200165,0.175615,0.340986,0.068807,0.000000,0.429265,0.107925,0.160852,0.214240,0.036933,0.233471,0.262186,0.124593,0.266240,0.068840,0.096171,0.179761,0.160145,0.238781,0.243788,0.123810,0.275831,0.395287,0.308475,0.055872,0.197862,0.131367,0.152449,0.084456,0.293293,0.056765,0.103536,0.326491
2,0.118076,1.000000,0.099097,0.107680,0.034279,0.152789,0.086705,0.078864,0.068940,0.092399,0.098726,0.074706,0.143962,0.088732,0.352885,0.067225,0.141859,0.111234,0.085395,0.023666,0.117823,0.043818,0.085681,0.142768,0.082198,0.415777,0.108961,0.039521,0.196305,0.183749,0.052282,0.159113,0.083984,0.071035,0.070759,0.041877,0.071331,0.050981,0.190476,0.171597,...,0.173430,0.110674,0.228069,0.163651,0.052568,0.121685,0.302777,0.035253,0.031939,0.107282,0.011763,0.172940,0.083593,0.241509,0.028323,0.221194,0.283037,0.141837,0.080040,0.171161,0.143195,0.029920,0.391046,0.099513,0.091941,0.054671,0.177199,0.374315,0.011658,0.065508,0.086927,0.259636,0.289092,0.318824,0.149105,0.186347,0.168034,0.106748,0.136796,0.080358
3,0.029097,0.099097,1.000000,0.252131,0.026893,0.062539,0.039767,0.089474,0.078162,0.037670,0.031866,0.086560,0.138187,0.040749,0.098841,0.057337,0.006849,0.027751,0.000000,0.039608,0.114870,0.027501,0.022252,0.044016,0.000000,0.115385,0.006513,0.013229,0.136596,0.063315,0.154689,0.071696,0.319455,0.095315,0.201552,0.067167,0.000000,0.053754,0.168958,0.296983,...,0.047419,0.157270,0.058609,0.045193,0.058653,0.075120,0.049184,0.011800,0.032073,0.057332,0.000000,0.271350,0.014727,0.027311,0.008080,0.205548,0.345030,0.084334,0.011482,0.138457,0.051355,0.156487,0.171978,0.013431,0.032057,0.033273,0.006673,0.143409,0.000000,0.034622,0.040918,0.019031,0.065417,0.055373,0.086503,0.018418,0.096993,0.109631,0.092574,0.018987
4,0.011628,0.107680,0.252131,1.000000,0.000000,0.045543,0.078812,0.095354,0.059498,0.053879,0.074209,0.038437,0.060181,0.034741,0.107286,0.059952,0.000000,0.014787,0.044219,0.031658,0.085692,0.014654,0.022396,0.142399,0.026656,0.068105,0.041645,0.089874,0.182678,0.048197,0.037467,0.084593,0.228098,0.206432,0.167468,0.080916,0.034698,0.069562,0.186854,0.142598,...,0.000000,0.041901,0.036034,0.065676,0.101574,0.066713,0.076241,0.025151,0.042725,0.068204,0.000000,0.119287,0.015694,0.000000,0.000000,0.121569,0.080093,0.053259,0.073420,0.067841,0.020524,0.156763,0.160368,0.093040,0.027331,0.000000,0.000000,0.040219,0.019493,0.024598,0.024226,0.050703,0.056561,0.107294,0.098892,0.000000,0.132900,0.142798,0.097066,0.015176
5,0.264677,0.034279,0.026893,0.000000,1.000000,0.202843,0.299619,0.163724,0.038474,0.153021,0.290192,0.101406,0.298501,0.203679,0.090818,0.198008,0.024542,0.211799,0.123906,0.163770,0.138529,0.310043,0.322358,0.105729,0.182711,0.093950,0.052063,0.275308,0.000000,0.102223,0.028427,0.111519,0.000000,0.003390,0.000000,0.000000,0.146963,0.184862,0.000000,0.003598,...,0.033272,0.028901,0.024854,0.132500,0.230388,0.000000,0.081344,0.313078,0.166869,0.263148,0.052910,0.000000,0.236461,0.061731,0.128287,0.109758,0.000000,0.191999,0.329955,0.112303,0.197299,0.063840,0.000000,0.195483,0.113108,0.193750,0.114039,0.062417,0.254115,0.303009,0.262547,0.048524,0.048312,0.022202,0.091910,0.066000,0.156172,0.115842,0.124297,0.26757

In [25]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(user_id, movie_id):
    
    #Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[user_id]
        
        #Get the user ratings for the movie in question
        m_ratings = r_matrix[movie_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating

In [26]:
score(cf_user_wmean)

1.0174483808407588

### Demographics

In [27]:
#Merge the original users dataframe with the training set 
merged_df = pd.merge(X_train, users)

merged_df.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,889,684,2,24,M,technician,78704
1,889,279,2,24,M,technician,78704
2,889,29,3,24,M,technician,78704
3,889,190,3,24,M,technician,78704
4,889,232,3,24,M,technician,78704


In [28]:
#Compute the mean rating of every movie by gender
gender_mean = merged_df[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [29]:
#Set the index of the users dataframe to the user_id
users = users.set_index('user_id')

In [30]:
#Gender Based Collaborative Filter using Mean Ratings
def cf_gender(user_id, movie_id):
    
    #Check if movie_id exists in r_matrix (or training set)
    if movie_id in r_matrix:
        #Identify the gender of the user
        gender = users.loc[user_id]['sex']
        
        #Check if the gender has rated the movie
        if gender in gender_mean[movie_id]:
            
            #Compute the mean rating given by that gender to the movie
            gender_rating = gender_mean[movie_id][gender]
        
        else:
            gender_rating = 3.0
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        gender_rating = 3.0
    
    return gender_rating

In [31]:
score(cf_gender)

1.0330308800874282

In [32]:
#Compute the mean rating by gender and occupation
gen_occ_mean = merged_df[['sex', 'rating', 'movie_id', 'occupation']].pivot_table(
    values='rating', index='movie_id', columns=['occupation', 'sex'], aggfunc='mean')

gen_occ_mean.head()

occupation administrator           artist  ... technician    writer          
sex                    F         M      F  ...          M         F         M
movie_id                                   ...                               
1                    4.0  4.222222   4.25  ...   4.200000  4.166667  3.142857
2                    3.0  3.750000    NaN  ...   2.714286  5.000000  2.666667
3                    3.5  2.500000    NaN  ...   4.000000       NaN  1.000000
4                    3.0  3.888889    NaN  ...   3.200000  4.250000  3.500000
5                    4.0  2.333333    NaN  ...   3.333333  4.000000  2.666667

[5 rows x 41 columns]

In [33]:
#Gender and Occupation Based Collaborative Filter using Mean Ratings
def cf_gen_occ(user_id, movie_id):
    
    #Check if movie_id exists in gen_occ_mean
    if movie_id in gen_occ_mean.index:
        
        #Identify the user
        user = users.loc[user_id]
        
        #Identify the gender and occupation
        gender = user['sex']
        occ = user['occupation']
        
        #Check if the occupation has rated the movie
        if occ in gen_occ_mean.loc[movie_id]:
            
            #Check if the gender has rated the movie
            if gender in gen_occ_mean.loc[movie_id][occ]:
                
                #Extract the required rating
                rating = gen_occ_mean.loc[movie_id][occ][gender]
                
                #Default to 3.0 if the rating is null
                if np.isnan(rating):
                    rating = 3.0
                
                return rating
            
    #Return the default rating    
    return 3.0

In [34]:
score(cf_gen_occ)

1.1391976012043645

## Model Based Approaches

In [36]:
!pip install surprise

     |████████████████████████████████| 11.8MB 4.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670954 sha256=6f39abc0ad3a5f6185c1e1dbde4936f0cebcbaa3cabca755d5323dc20b606beb
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [39]:
#Import the required classes and methods from the surprise library
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate #replace for evaluate()

#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter
data = Dataset.load_from_df(ratings, reader)

#Define the algorithm object; in this case kNN
knn = KNNBasic()

#Evaluate the performance in terms of RMSE
#evaluate(knn, data, measures=['RMSE'])


# Run 5-fold cross-validation and print results
cross_validate(knn, data, measures=['RMSE'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9864  0.9925  0.9892  0.9893  0.0025  
Fit time          0.44    0.46    0.49    0.46    0.02    
Test time         6.12    6.28    6.33    6.24    0.09    


{'fit_time': (0.44236111640930176, 0.4626145362854004, 0.48784852027893066),
 'test_rmse': array([0.98635249, 0.99248302, 0.98919075]),
 'test_time': (6.11767578125, 6.2827231884002686, 6.328152179718018)}

In [40]:
#Import SVD
from surprise import SVD

#Define the SVD algorithm object
svd = SVD()

#Evaluate the performance in terms of RMSE
#evaluate(svd, data, measures=['RMSE'])

# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9441  0.9451  0.9458  0.9450  0.0007  
Fit time          4.55    4.63    4.72    4.63    0.07    
Test time         0.42    0.26    0.40    0.36    0.07    


{'fit_time': (4.5466179847717285, 4.634291172027588, 4.718250274658203),
 'test_rmse': array([0.94408006, 0.94509942, 0.94584326]),
 'test_time': (0.41843652725219727, 0.2573983669281006, 0.39621663093566895)}